# Classifying Red Wine Quality Types using a Multi-Layer Perceptron


*   We will aim to build an end-to-end deep learning model for wine quality classification. 
* However, this will be benchmarked against a model trained on a preprocessed red wine quality dataset that has undergone feature selection using ANOVA.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, Dropout
from tensorflow.keras.initializers import GlorotNormal
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
!pip install keras-tuner
from kerastuner.tuners import Hyperband
from kerastuner import HyperModel
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
%matplotlib inline
%load_ext tensorboard

In [2]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load Red Wine Data
wine_df = pd.read_csv("/content/drive/My Drive/Business Analytics Stuff/BT2101 Group Project/winequality-red.csv")

# 1. Data Preprocessing

## 1.1 Preprocessing & Feature Engineering

In [4]:
# Creating the Binary Target Variable
wine_df["good_quality"] = wine_df["quality"] >= 7

# Target Variable
target = wine_df.loc[:, 'good_quality']
target = target.astype(int)  # 1 = good quality & 0 = other quality
target

# Interaction Terms
final_df = wine_df.drop(columns=['quality', 'good_quality']).copy()
final_df['total acidity'] = final_df['fixed acidity'] + final_df['volatile acidity']
final_df['pH_sulphate'] = final_df['pH'] * final_df['sulphates']

# View final_df
final_df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,total acidity,pH_sulphate
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,8.10,1.9656
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,8.68,2.1760
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,8.56,2.1190
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,11.48,1.8328
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,8.10,1.9656
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,8.06,1.9656
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,8.50,1.5180
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7.95,1.5933
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,8.38,1.9152
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,8.00,2.6800


In [5]:
# 13 Features, hence 13 input neurons
len(final_df.columns)

13

In [6]:
# Train and Test sets
X_train, X_test_temp, y_train, y_test_temp = train_test_split(final_df, target, test_size=0.40)

# Test and Validation sets
X_test, X_val, y_test, y_val = train_test_split(X_test_temp, y_test_temp, test_size=0.50)

# 2. End to End Classifier
## 2.1 Baseline NN
### 2.1.1 Baseline NN Classifier without Tuning or Weights

In [7]:
# Model
def build_model():
  model = Sequential()
  model.add(Dense(30, input_dim=13, activation='relu'))
  model.add(Dense(60, activation='relu'))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

### Cross validated results
Scale values to [-1, 1] instead of [0, 1] for more efficient backpropagation 

(see https://stats.stackexchange.com/questions/249378/is-scaling-data-0-1-necessary-when-batch-normalization-is-used)

In [8]:
# Create Pipeline 1 with Standard Scaler
pipe = []
pipe.append(('standardize', StandardScaler()))
pipe.append(('mlp', KerasClassifier(build_fn=build_model, epochs=30, batch_size=15, verbose=0)))
pipeline = Pipeline(pipe)

In [9]:
# Train and get Cross Val Accuraccy
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, final_df, target ,cv=kfold)

In [10]:
print("Baseline NN Classifier has a mean cross val accuracy score of {:.2f}%".format(results.mean()*100))

Baseline NN Classifier has a mean cross val accuracy score of 89.56%


### Train-Test Results

In [11]:
# Get F1-score
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [12]:
# Classification report for Baseline NN Classifier 
pd.DataFrame(classification_report(y_test, y_pred, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.940767,0.606061,0.90625,0.773414,0.902066
recall,0.954064,0.540541,0.90625,0.747302,0.906250
f1-score,0.947368,0.571429,0.90625,0.759398,0.903900
support,283.000000,37.000000,0.90625,320.000000,320.000000


### 2.1.2 Baseline NN with Class Weights to correct for Imbalanced Dataset

In [13]:
# Scale & Instantiate Sample Weights
weights = {0:1, 1:10}
std_scaler = StandardScaler()
X_train_scaled = std_scaler.fit_transform(X_train, y_train)
X_val_scaled = std_scaler.fit_transform(X_val, y_val)
X_test_scaled = std_scaler.fit_transform(X_test, y_test)

# Call backs for early stopping
my_callbacks = [
                EarlyStopping(monitor='val_accuracy', patience=50, mode='max'),
                TensorBoard(log_dir='./logs')
                ]

weighted_classifier = build_model()
weighted_classifier.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), class_weight=weights, epochs=200, batch_size=16, callbacks=my_callbacks)

Epoch 1/200
 1/60 [..............................] - ETA: 0s - loss: 1.5593 - accuracy: 0.3750WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
60/60 [==============================] - 0s 6ms/step - loss: 1.2576 - accuracy: 0.3942 - val_loss: 0.7283 - val_accuracy: 0.6344
Epoch 2/200
60/60 [==============================] - 0s 4ms/step - loss: 1.0200 - accuracy: 0.7059 - val_loss: 0.6795 - val_accuracy: 0.6625
Epoch 3/200
60/60 [==============================] - 0s 3ms/step - loss: 0.9271 - accuracy: 0.7414 - val_loss: 0.7515 - val_accuracy: 0.6313
Epoch 4/200
60/60 [==============================] - 0s 3ms/step - loss: 0.8682 - accuracy: 0.7424 - val_loss: 0.6793 - val_accuracy: 0.6656
Epoch 5/200
60/60 [==============================] - 0s 3ms/step - loss: 0.8

### Visualising the training process

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs

In [15]:
# Predict
y_pred2 = weighted_classifier.predict(X_test_scaled) > 0.5

In [16]:
# Classification Report for Baseline NN Classifier with Weighted Classes
pd.DataFrame(classification_report(y_test, y_pred2, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.942652,0.512195,0.8875,0.727424,0.892881
recall,0.929329,0.567568,0.8875,0.748448,0.887500
f1-score,0.935943,0.538462,0.8875,0.737202,0.889984
support,283.000000,37.000000,0.8875,320.000000,320.000000


## 2.2 Hyperparameter Tuning of Baseline NN Classifier


1.   No. of hidden units
2.   Learning Rate
3.   Mini-Batch size
4.   No. of layers

In [17]:
class tuned_classifier(HyperModel):
  def __init__(self, input_size=13):
    self.input_size = input_size
  
  def build(self, hp):
    model = Sequential()
    # Input and First Layer
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=256, step=32, default=128), input_dim=self.input_size, activation=hp.Choice('dense_activation_1', values=['relu', 'tanh'], default='relu')))
    # Second Layer
    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=256, step=32, default=128), activation=hp.Choice('dense_activation_2', values=['relu', 'tanh'], default='relu')))
    # Third Layer
    model.add(Dense(units=hp.Int('units_3', min_value=32, max_value=256, step=32, default=128), activation=hp.Choice('dense_activation_3', values=['relu', 'tanh'], default='relu')))
    # Output Layer
    model.add(Dense(1, activation='sigmoid'))
    # Compile Model
    model.compile(loss='binary_crossentropy', optimizer=Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3)), metrics=['accuracy'])
    return model


In [18]:
# Instantiate Hyper Model
hypermodel = tuned_classifier()

In [19]:
# Instantiate Tuner
tuner = Hyperband(
    hypermodel,
    max_epochs=10,
    objective='val_accuracy',
    executions_per_trial=2,
    seed=42,
    project_name='red_wine_classification'
)

INFO:tensorflow:Reloading Oracle from existing project ./red_wine_classification/oracle.json
INFO:tensorflow:Reloading Tuner from ./red_wine_classification/tuner0.json


In [20]:
tuner.search_space_summary()

Search space summary
Default search space size: 7
units_1 (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
dense_activation_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
units_2 (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
dense_activation_2 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
units_3 (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
dense_activation_3 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
learning_rate (Float)
{'default': 0.001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [21]:
# Randomised Search
tuner.search(X_train, y_train, epochs=5, validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [22]:
# Retrieve the best model.
best_hps = tuner.get_best_models(num_models=1)[0]

# Show summary of Best Model
best_hps.get_config()

{'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 13),
    'dtype': 'float32',
    'name': 'dense_input',
    'ragged': False,
    'sparse': False}},
  {'class_name': 'Dense',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'batch_input_shape': (None, 13),
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'dtype': 'float32',
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'kernel_regularizer': None,
    'name': 'dense',
    'trainable': True,
    'units': 128,
    'use_bias': True}},
  {'class_name': 'Dense',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'dtype': 'float32',
    'kernel_constraint': None,
    'kernel_initializer': 

In [23]:
best_hps.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1792      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 224)               57568     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 225       
Total params: 92,609
Trainable params: 92,609
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Build optimal model using best hyperparameters
hps_best_params = tuner.get_best_hyperparameters(num_trials = 1)[0]
optimal_classifier = tuner.hypermodel.build(hps_best_params)

In [25]:
# Create call backs
my_callbacks_opt = [EarlyStopping(monitor='val_accuracy', patience=50, mode='max')]

# Train model
optimal_classifier.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), class_weight=weights, epochs=200, batch_size=16, callbacks=my_callbacks_opt)

Epoch 1/200
60/60 [==============================] - 0s 5ms/step - loss: 1.1842 - accuracy: 0.6663 - val_loss: 0.6825 - val_accuracy: 0.6406
Epoch 2/200
60/60 [==============================] - 0s 3ms/step - loss: 0.8886 - accuracy: 0.7278 - val_loss: 0.5155 - val_accuracy: 0.7281
Epoch 3/200
60/60 [==============================] - 0s 3ms/step - loss: 0.8231 - accuracy: 0.7518 - val_loss: 0.5600 - val_accuracy: 0.7312
Epoch 4/200
60/60 [==============================] - 0s 4ms/step - loss: 0.7249 - accuracy: 0.7633 - val_loss: 0.3607 - val_accuracy: 0.8062
Epoch 5/200
60/60 [==============================] - 0s 4ms/step - loss: 0.7371 - accuracy: 0.7967 - val_loss: 0.3966 - val_accuracy: 0.8000
Epoch 6/200
60/60 [==============================] - 0s 4ms/step - loss: 0.6586 - accuracy: 0.8019 - val_loss: 0.5956 - val_accuracy: 0.7812
Epoch 7/200
60/60 [==============================] - 0s 4ms/step - loss: 0.6308 - accuracy: 0.8040 - val_loss: 0.4719 - val_accuracy: 0.7875
Epoch 8/200
6

In [26]:
# Optimal model predictions
y_pred3 = optimal_classifier.predict(X_test_scaled) > 0.5

In [27]:
# Classification Report of Tuned Baseline NN Classifier
pd.DataFrame(classification_report(y_test, y_pred3, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.958491,0.472727,0.875,0.715609,0.902324
recall,0.897527,0.702703,0.875,0.800115,0.875000
f1-score,0.927007,0.565217,0.875,0.746112,0.885175
support,283.000000,37.000000,0.875,320.000000,320.000000


## 2.3 Tuned NN Classifier with Dropout Regularisation & Batch Normalisation
### 2.3.1 with Dropout
We will set keep_prob = 0.8 or dropout_rate = 0.2 as a baseline. 

In [28]:
def build_dropout_model():
  model = Sequential()
  model.add(Dense(128, input_dim=13, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(224, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [29]:
# Create Pipeline for Dropout
pipe_dropout = []
pipe_dropout.append(('standardize', StandardScaler()))
pipe_dropout.append(('mlp', KerasClassifier(build_fn=build_dropout_model, epochs=100, batch_size=16, verbose=0)))
pipeline_dropout = Pipeline(pipe_dropout)

In [30]:
# Train and Predict
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results_dropout = cross_val_score(pipeline_dropout, final_df, target, cv=kfold)

In [31]:
print("Baseline NN Classifier with Dropout has a mean cross val accuracy of {:.2f}%".format(results_dropout.mean()*100))

Baseline NN Classifier with Dropout has a mean cross val accuracy of 89.31%


In [32]:
# Dropout model & F1
dropout_model = build_dropout_model()

# Create call backs
my_callbacks_dropout = [EarlyStopping(monitor='val_accuracy', patience=50, mode='max')]

# Train model
dropout_model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), class_weight=weights, epochs=200, batch_size=16, callbacks=my_callbacks_dropout)

Epoch 1/200
60/60 [==============================] - 0s 5ms/step - loss: 1.1192 - accuracy: 0.7414 - val_loss: 0.6370 - val_accuracy: 0.6844
Epoch 2/200
60/60 [==============================] - 0s 3ms/step - loss: 0.9196 - accuracy: 0.7258 - val_loss: 0.5930 - val_accuracy: 0.6906
Epoch 3/200
60/60 [==============================] - 0s 4ms/step - loss: 0.8765 - accuracy: 0.7456 - val_loss: 0.5904 - val_accuracy: 0.6906
Epoch 4/200
60/60 [==============================] - 0s 4ms/step - loss: 0.8376 - accuracy: 0.7351 - val_loss: 0.6638 - val_accuracy: 0.7000
Epoch 5/200
60/60 [==============================] - 0s 4ms/step - loss: 0.8378 - accuracy: 0.7466 - val_loss: 0.5872 - val_accuracy: 0.7000
Epoch 6/200
60/60 [==============================] - 0s 4ms/step - loss: 0.7651 - accuracy: 0.7508 - val_loss: 0.4837 - val_accuracy: 0.7406
Epoch 7/200
60/60 [==============================] - 0s 4ms/step - loss: 0.7722 - accuracy: 0.7487 - val_loss: 0.4878 - val_accuracy: 0.7500
Epoch 8/200
6

In [33]:
# Get predictions and F1 score
y_pred4=dropout_model.predict(X_test_scaled) > 0.5

# F1 score
pd.DataFrame(classification_report(y_test, y_pred4, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.942238,0.488372,0.88125,0.715305,0.889760
recall,0.922261,0.567568,0.88125,0.744915,0.881250
f1-score,0.932143,0.525000,0.88125,0.728571,0.885067
support,283.000000,37.000000,0.88125,320.000000,320.000000


#### 2.3.2 with Batch Normalisation & Dropout
Best Results:

---
1. Dropout = 0.2: Poorer results | F1 Macro: 0.761905	| Accuracy: 0.8625 
2. Dropout = 0.15 Best results | F1 Macro: 0.787359 | Accuracy: 0.8906 | CV Acc: 0.9012
3. Dropout = 0.12 Better results | F1 Macro: 0.7431 | Accuracy: 0.8656 | CV Acc: 0.8999
---
Since batch norm introduces noise and thus a slight regularisation effect whilst minimising covariate shift. We will reduce the dropout_rate to retain more information. (Tune with HyperModel)

In [34]:
def build_bn_dropout_model(dropout=0.15):
  model = Sequential()

  model.add(Dense(128, input_dim=13, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout))

  model.add(Dense(256, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout))

  model.add(Dense(224, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout))

  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [35]:
# Create Pipeline for BatchNorm
pipe_bn_dropout = []
pipe_bn_dropout.append(('standardize', StandardScaler()))
pipe_bn_dropout.append(('mlp', KerasClassifier(build_fn=build_bn_dropout_model, epochs=200, batch_size=16, verbose=0)))
pipeline_bn_dropout = Pipeline(pipe_bn_dropout)

In [36]:
# Train and Predict
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results_bnorm = cross_val_score(pipeline_bn_dropout, final_df, target, cv=kfold)

In [37]:
print("Baseline NN Classifier with Batch Normalisation has a mean cross val accuracy of {:.2f}%".format(results_bnorm.mean()*100))

Baseline NN Classifier with Batch Normalisation has a mean cross val accuracy of 90.31%


In [38]:
# Dropout model & F1
bn_dropout_model = build_bn_dropout_model()

# Create call backs
my_callbacks_bn_dropout = [
                           EarlyStopping(monitor='val_accuracy', patience=50, mode='max'),
                           TensorBoard(log_dir='./logs')
                           ]

# Train model
bn_dropout_model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), class_weight=weights, epochs=200, batch_size=16, callbacks=my_callbacks_bn_dropout)

Epoch 1/200
60/60 [==============================] - 2s 27ms/step - loss: 1.3603 - accuracy: 0.6142 - val_loss: 0.6304 - val_accuracy: 0.6719
Epoch 2/200
60/60 [==============================] - 0s 6ms/step - loss: 1.1369 - accuracy: 0.6674 - val_loss: 0.5606 - val_accuracy: 0.7469
Epoch 3/200
60/60 [==============================] - 0s 6ms/step - loss: 1.0406 - accuracy: 0.6966 - val_loss: 0.5344 - val_accuracy: 0.7437
Epoch 4/200
60/60 [==============================] - 0s 6ms/step - loss: 0.9537 - accuracy: 0.7195 - val_loss: 0.5187 - val_accuracy: 0.7625
Epoch 5/200
60/60 [==============================] - 0s 6ms/step - loss: 0.8772 - accuracy: 0.7602 - val_loss: 0.5912 - val_accuracy: 0.7281
Epoch 6/200
60/60 [==============================] - 0s 6ms/step - loss: 0.8127 - accuracy: 0.7487 - val_loss: 0.5866 - val_accuracy: 0.7188
Epoch 7/200
60/60 [==============================] - 0s 6ms/step - loss: 0.7743 - accuracy: 0.7477 - val_loss: 0.5711 - val_accuracy: 0.7688
Epoch 8/200


In [ ]:
# Load tensorboard
%tensorboard --logdir logs

In [40]:
# Get predictions and F1 score
y_pred5 = bn_dropout_model.predict(X_test_scaled) > 0.5

# F1 score
pd.DataFrame(classification_report(y_test, y_pred5, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.965251,0.459016,0.86875,0.712134,0.906718
recall,0.883392,0.756757,0.86875,0.820074,0.868750
f1-score,0.922509,0.571429,0.86875,0.746969,0.881916
support,283.000000,37.000000,0.86875,320.000000,320.000000
